In [ ]:
!pip install pennylane

In [117]:
import pennylane as qml
import numpy as np
from random import randint

NUM_TRIALS = 1000

# 4 ancilla qubits for two parity boxes operations
dev = qml.device("default.qubit", wires=8, shots=1)

@qml.qnode(dev)
def prepare_and_measure(alice_row, bob_col):
    """
    Prepare the quantum state and perform measurements based on
    Alice's row and Bob's column choices.
    """
    # Create two Bell pairs (entangled qubits)
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=2)
    qml.CNOT(wires=[2, 3])

    # Alice's operations for row 2 (Two parity box)
    if alice_row == 2:
        qml.CNOT(wires=[0, 4])
        qml.CNOT(wires=[2, 4])
        qml.Hadamard(wires=0)
        qml.CNOT(wires=[0, 5])
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=2)
        qml.CNOT(wires=[2, 5])
        qml.Hadamard(wires=2)

    # Bob's operations for column 2 (Two parity box)
    if bob_col == 2:
        qml.Hadamard(wires=1)
        qml.CNOT(wires=[1, 6])
        qml.Hadamard(wires=1)
        qml.CNOT(wires=[3, 6])
        qml.CNOT(wires=[1, 7])
        qml.Hadamard(wires=3)
        qml.CNOT(wires=[3, 7])
        qml.Hadamard(wires=3)

    # Alice's measurements based on her row
    alice_measurements = []
    if alice_row == 0:
        alice_measurements = [qml.sample(qml.PauliZ(2)), qml.sample(qml.PauliX(0))]
    elif alice_row == 1:
        alice_measurements = [qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliX(2))]
    elif alice_row == 2:
        alice_measurements = [qml.sample(-qml.PauliZ(4)), qml.sample(-qml.PauliZ(5))]

    # Bob's measurements based on his column
    bob_measurements = []
    if bob_col == 0:
        bob_measurements = [qml.sample(qml.PauliZ(3)), qml.sample(qml.PauliZ(1))]
    elif bob_col == 1:
        bob_measurements = [qml.sample(qml.PauliX(1)), qml.sample(qml.PauliX(3))]
    elif bob_col == 2:
        bob_measurements = [qml.sample(qml.PauliZ(6)), qml.sample(qml.PauliZ(7))]

    return alice_measurements, bob_measurements

def run_experiment(num_trials=NUM_TRIALS):
    """
    Run the quantum game experiment and collect statistics.
    """
    wins = {(a, b): 0 for a in range(3) for b in range(3)}
    counts = {(a, b): 0 for a in range(3) for b in range(3)}
    total_wins = 0

    for _ in range(num_trials):
        alice_row = randint(0, 2)
        bob_col = randint(0, 2)

        # Run quantum circuit
        am, bm = prepare_and_measure(alice_row, bob_col)

        # Process results
        am = np.array(am)
        bm = np.array(bm)

        # Append products to complete measurement arrays
        am = np.append(am, np.prod(am))
        bm = np.append(bm, -np.prod(bm))

        # Check win condition
        if (am[bob_col] == bm[alice_row] and
            np.prod(am) == 1 and
            np.prod(bm) == -1):
            wins[(alice_row, bob_col)] += 1
            total_wins += 1

        counts[(alice_row, bob_col)] += 1

    # Print statistics
    print(f"=== Quantum Game Statistics (Trials: {num_trials}) ===")
    print("Alice Row | Bob Column | Wins/Trials | Win Rate (%)")
    print("-" * 50)

    for a in range(3):
        for b in range(3):
            win_rate = (wins[(a, b)] / counts[(a, b)]) * 100 if counts[(a, b)] > 0 else 0
            print(f"    {a+1}     |     {b+1}      |  {wins[(a, b)]}/{counts[(a, b)]}  | {win_rate:.2f}%")

    # Overall statistics
    overall_win_rate = (total_wins / num_trials) * 100
    print("-" * 50)
    print(f"Overall Results: {total_wins}/{num_trials} wins ({overall_win_rate:.2f}%)")

if __name__ == "__main__":
    run_experiment()

=== Quantum Game Statistics (Trials: 1000) ===
Alice Row | Bob Column | Wins/Trials | Win Rate (%)
--------------------------------------------------
    1     |     1      |  120/120  | 100.00%
    1     |     2      |  123/123  | 100.00%
    1     |     3      |  115/115  | 100.00%
    2     |     1      |  110/110  | 100.00%
    2     |     2      |  105/105  | 100.00%
    2     |     3      |  111/111  | 100.00%
    3     |     1      |  104/104  | 100.00%
    3     |     2      |  108/108  | 100.00%
    3     |     3      |  104/104  | 100.00%
--------------------------------------------------
Overall Results: 1000/1000 wins (100.00%)
